# 한국 거래소에서 시가총액 데이터 수집
* KRX에서 상장 기업의 시가총액 데이터를 기간별로 수집하는 방법에 대해서 알아보겠습니다. 
* selenium, BeautifulSoup, requests, json 라이브러리를 사용하여 진행하겠습니다. 

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd 
import requests
import json
import time

* Chrome를 실제로 운전하게 될 driver라는 웹 드라이버 객체를 생성합니다. 
    > Chrome, Firefox, PhantomJS등 각각의 브라우저마다 드라이버가 다르고, 호출 함수도 다릅니다. 
* 드라이버에게 파이썬 코드로 우리가 웹에서 하는 행동을 지시할 수 있습니다. 
* chromedriver.exe 파일이 저장되어 있는 경로를 Chrome() 메소드에 전달 

In [2]:
driver = webdriver.Chrome("C:\chromedriver\chromedriver.exe")

* get() 메소드를 사용하여 사용자가 원하는 주소로 이동합니다.
* 이동 뒤에 sleep() 함수를 사용하여 페이지가 완전히 로딩될 때까지 기다립니다. 

In [3]:
# 종합 시황 다운로드 받기 
driver.get('http://marketdata.krx.co.kr/mdi#document=0101')
# 5초간 기다림
time.sleep(5)

* find_element_by_css_selector 메소드는 selector에서 복사한 값을 기준으로 해당 지정한 곳의 정보를 찾습니다.
* 찾고 난 뒤에 텍스트 상자라면 clear()를 사용하여 값을 삭제할수도, 버튼이라면 click()를 사용하여 실제 클릭을 진행할 수 있습니다.
* 아래 예제는 문서 3개를 다운로드를 자동으로 받는 예제입니다. click() 함수를 사용하여 자동으로 다운로드 합니다.

In [4]:
driver.find_element_by_css_selector("#c9f0f895fb98ab9159f51fd0297e236d > div.coreboard-data.design-board-normal > table > tbody > tr:nth-child(1) > td.attach-td").click()
driver.find_element_by_css_selector("#c9f0f895fb98ab9159f51fd0297e236d > div.coreboard-data.design-board-normal > table > tbody > tr:nth-child(1) > td.attach-td > div > div > ul > li > a").click()

In [5]:
driver.find_element_by_css_selector("#c9f0f895fb98ab9159f51fd0297e236d > div.coreboard-data.design-board-normal > table > tbody > tr:nth-child(2) > td.attach-td > div > button").click()           
driver.find_element_by_css_selector("#c9f0f895fb98ab9159f51fd0297e236d > div.coreboard-data.design-board-normal > table > tbody > tr:nth-child(2) > td.attach-td > div > div > ul > li > a").click()      

In [6]:
driver.find_element_by_css_selector("#c9f0f895fb98ab9159f51fd0297e236d > div.coreboard-data.design-board-normal > table > tbody > tr:nth-child(3) > td.attach-td > div > button").click()           
driver.find_element_by_css_selector("#c9f0f895fb98ab9159f51fd0297e236d > div.coreboard-data.design-board-normal > table > tbody > tr:nth-child(3) > td.attach-td > div > div > ul > li > a").click()       

* 조회 기간의 글자를 clear() 메소드를 사용하여 깨끗하기 지웁니다. 
* 그리고 send_keys() 메소드를 사용하여 내가 원하는 날짜 값을 전달합니다. 

In [7]:
driver.find_element_by_css_selector('#fromdate1679091c5a880faf6fb5e6087eb1b2dc').clear()

In [8]:
driver.find_element_by_css_selector('#fromdate1679091c5a880faf6fb5e6087eb1b2dc').send_keys('20190628')

* 자동화를 진행하게 된다면 20190628처럼 사람이 직접 입력하지 않고 자동으로 만들어야 합니다. 
* 우리가 이전 시간에 배웠던 datetime 라이브러리의 date 클래스의 today() 메소드를 사용하여 현재 날짜를 자동으로 가져옵니다.

In [9]:
import datetime

In [10]:
datetime.date.today()

datetime.date(2019, 7, 2)

* str() 함수를 사용하면 문자열로 변경할 수 있고, 문자열 메소드인 replace()를 사용하면 -를 제거할 수 있습니다.

In [11]:
str(datetime.date.today()).replace("-", "")

'20190702'

* date객체에서 strftime() 메소드를 사용하면 문자열 변환을 포맷에 맞게 출력할 수 있습니다.

In [12]:
today = datetime.date.today()

* %Y : year
* %m : Month
* %d : day

In [13]:
today.strftime("%Y%m%d")

'20190702'

# 시가총액 데이터 다운로드 받기 

* 시가 총액을 다운로드 받을 수 있는 사이트로 이동합니다.
* refresh() 함수는 페이지를 새로고침 합니다. 

In [14]:
driver.get('http://marketdata.krx.co.kr/mdi#document=040402')
driver.refresh()
# 5초간 페이지에서 기다립니다. 
time.sleep(5)

* code 값은 매번 값이 변경되고 지금 code값을 설정하여도 다시 실행하면 동작하지 않은 것을 확인할 수 있습니다.
* form 태그의 data-code 값은 웹 페이지에 접속하고 자바스크립트가 실행되어야 최종적으로 알 수 있는 데이터이기 때문에 selenium를 사용합니다.
* form 태그의 action 값과 method값으로 해당 값을 찾고 'data-code'의 값을 추출합니다.
* payload라는 dict는 post 방식으로 krx하고 통신을 하기 위한 요청사항 정보입니다. 이 정보를 바탕으로 서버에서 정보를 전달합니다.
* 이때 웹 페이지에 있는 data-code 값을 확인하여 정상적인 data-code면 제대로 된 값을 전달합니다. 

In [15]:
bs = BeautifulSoup(driver.page_source, "html.parser") 
rt2 = bs.find("form", {"action" : "/contents/MKD/99/MKD99000001.jspx", 
                       "method" : "post", "data-bld":"MKD/04/0404/04040200/mkd04040200_01"})


payload = {'market_gubun' : 'ALL',
           'sect_tp_cd' : 'ALL',
           'schdate' : '20190627',
           'pagePath' : '/contents/MKD/04/0404/04040200/MKD04040200.jsp'
           }

payload['code'] = rt2['data-code']

* curPage는 시가총액의 페이지 번호를 의미합니다. 아래 예제는 1 페이지를 설정합니다. 

In [16]:
payload['curPage'] = 1

In [17]:
ret= requests.post("http://marketdata.krx.co.kr/contents/MKD/99/MKD99000001.jspx",  data=payload)

In [18]:
ret.text

'{"시가총액 상하위":[{"rn":"1","isu_cd":"005930","kor_shrt_isu_nm":"삼성전자","isu_cur_pr":"46,500","fluc_tp_cd":"1","prv_dd_cmpr":"800","updn_rate":"1.8","isu_tr_vl":"12,603,534","isu_tr_amt":"583,888,994,023","opnprc":"46,000","hgprc":"46,600","lwprc":"45,750","cur_pr_tot_amt":"277,594,888,575,000","tot_amt_per":"16.65","lst_stk_vl":"5,969,782,550","f1":"3,411,981,118","f2":"57.15","totCnt":"2398"},{"rn":"2","isu_cd":"000660","kor_shrt_isu_nm":"SK하이닉스","isu_cur_pr":"70,200","fluc_tp_cd":"1","prv_dd_cmpr":"1,300","updn_rate":"1.9","isu_tr_vl":"5,361,201","isu_tr_amt":"378,736,507,000","opnprc":"70,100","hgprc":"71,600","lwprc":"69,700","cur_pr_tot_amt":"51,105,766,023,000","tot_amt_per":"3.07","lst_stk_vl":"728,002,365","f1":"367,493,736","f2":"50.48","totCnt":""},{"rn":"3","isu_cd":"005935","kor_shrt_isu_nm":"삼성전자우","isu_cur_pr":"38,000","fluc_tp_cd":"1","prv_dd_cmpr":"100","updn_rate":"0.3","isu_tr_vl":"1,399,856","isu_tr_amt":"53,134,817,735","opnprc":"37,600","hgprc":"38,150","lwprc":"37,600

* 시가총액 데이터를 selenium를 사용하여 데이터를 수집하면 자바스크립트가 실행되고, 이미지가 실행되는 등 속도가 많이 느립니다. 
* 그래서 requests의 post 방식은 값을 빠르게 전달받기 위해서 사용합니다. 
* 아래 for문을 300번까지 돌린 이유는 데이터가 없다면 크기를 확인하고(100) 이하면 종료 로직이 들어 있기 때문에 크게 잡았습니다.
* 종목은 수시로 변경될 수 있기 때문에 숫자를 딱 정하는 것보다 약간 더 크게 만들어 로직을 넣는 것을 추천합니다.

In [20]:
rt_list = []
for cnt in range(1,300):
    if(cnt % 10 == 0):
        print (cnt)
    payload['curPage'] = cnt
    rt3 = requests.post("http://marketdata.krx.co.kr/contents/MKD/99/MKD99000001.jspx",  data=payload)
    if(len(rt3.text) > 100):
        rt_list.append(rt3.text)
    else:
        break

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240


* rt_list의 원소 하나하나는 문자열로 이루어져 있습니다. 눈으로 확인하면 dict 형태이지만(서버에선 json형식으로 전달) 문자열이 때문에 json 라이브러리의 힘을 빌려야 합니다.

In [21]:
type(rt_list[0])

str

* json의 loads() 함수를 사용하여 문자열을 dict 형태로 변환합니다. 그리고 '시가총액 상하위' 키를 가지고 있는 값을 추출하여 DataFrame을 생성합니다.


In [22]:
pd.DataFrame(json.loads(rt_list[0])['시가총액 상하위'])

,cur_pr_tot_amt,f1,f2,fluc_tp_cd,hgprc,isu_cd,isu_cur_pr,isu_tr_amt,isu_tr_vl,kor_shrt_isu_nm,lst_stk_vl,lwprc,opnprc,prv_dd_cmpr,rn,totCnt,tot_amt_per,updn_rate
0,"277,594,888,575,000","3,411,981,118",57.15,1,"46,600",005930,"46,500","583,888,994,023","12,603,534",삼성전자,"5,969,782,550","45,750","46,000",800,1,2398,16.65,1.8
1,"51,105,766,023,000","367,493,736",50.48,1,"71,600",000660,"70,200","378,736,507,000","5,361,201",SK하이닉스,"728,002,365","69,700","70,100","1,300",2,,3.07,1.9
2,"31,269,694,600,000","762,330,049",92.64,1,"38,150",005935,"38,000","53,134,817,735","1,399,856",삼성전자우,"822,886,700","37,600","37,600",100,3,,1.88,0.3
3,"30,020,380,273,500","94,615,472",44.28,2,"142,500",005380,"140,500","48,441,831,000","343,337",현대차,"213,668,187","139,500","141,000","1,000",4,,1.80,-0.7
4,"26,179,130,280,000","27,239,509",21.23,2,"209,000",068270,"204,000","125,509,765,000","609,613",셀트리온,"128,329,070","203,000","207,500","6,500",5,,1.57,-3.1
5,"24,672,023,878,500","27,397,742",38.81,1,"350,000",051910,"349,500","59,918,992,000","172,767",LG화학,"70,592,343","343,000","346,000","2,000",6,,1.48,0.6
6,"22,444,726,437,000","45,640,625",47.89,1,"236,000",012330,"235,500","48,290,490,000","206,434",현대모비스,"95,306,694","228,000","229,000","2,500",7,,1.35,1.1
7,"21,410,111,353,050","319,203,336",67.31,1,"45,200",055550,"45,150","45,363,365,050","1,011,900",신한지주,"474,199,587","44,150","44,200",550,8,,1.28,1.2
8,"21,317,181,157,500","47,158,312",54.09,1,"245,500",005490,"244,500","42,601,962,500","174,978",POSCO,"87,186,835","240,500","242,500","1,500",9,,1.28,0.6
9,"20,841,975,000,000","5,745,073",8.68,1,"319,000",207940,"315,000","17,875,057,500","56,497",삼성바이오로직스,"66,165,000","312,000","313,500","1,500",10,,1.25,0.5


* 이 작업을 for문을 돌려서 concat() 함수(DataFrame을 row단위로 합쳐주는 역할) 호출하여 DataFrame을 total_df라는 하나의 데이터 합칩니다.

In [23]:
total_df = pd.DataFrame()
for x in rt_list:
    rt = json.loads(x)
    total_df = pd.concat([total_df, pd.DataFrame(rt['시가총액 상하위'])])

In [24]:
total_df

,cur_pr_tot_amt,f1,f2,fluc_tp_cd,hgprc,isu_cd,isu_cur_pr,isu_tr_amt,isu_tr_vl,kor_shrt_isu_nm,lst_stk_vl,lwprc,opnprc,prv_dd_cmpr,rn,totCnt,tot_amt_per,updn_rate
0,"277,594,888,575,000","3,411,981,118",57.15,1,"46,600",005930,"46,500","583,888,994,023","12,603,534",삼성전자,"5,969,782,550","45,750","46,000",800,1,2398,16.65,1.8
1,"51,105,766,023,000","367,493,736",50.48,1,"71,600",000660,"70,200","378,736,507,000","5,361,201",SK하이닉스,"728,002,365","69,700","70,100","1,300",2,,3.07,1.9
2,"31,269,694,600,000","762,330,049",92.64,1,"38,150",005935,"38,000","53,134,817,735","1,399,856",삼성전자우,"822,886,700","37,600","37,600",100,3,,1.88,0.3
3,"30,020,380,273,500","94,615,472",44.28,2,"142,500",005380,"140,500","48,441,831,000","343,337",현대차,"213,668,187","139,500","141,000","1,000",4,,1.80,-0.7
4,"26,179,130,280,000","27,239,509",21.23,2,"209,000",068270,"204,000","125,509,765,000","609,613",셀트리온,"128,329,070","203,000","207,500","6,500",5,,1.57,-3.1
5,"24,672,023,878,500","27,397,742",38.81,1,"350,000",051910,"349,500","59,918,992,000","172,767",LG화학,"70,592,343","343,000","346,000","2,000",6,,1.48,0.6
6,"22,444,726,437,000","45,640,625",47.89,1,"236,000",012330,"235,500","48,290,490,000","206,434",현대모비스,"95,306,694","228,000","229,000","2,500",7,,1.35,1.1
7,"21,410,111,353,050","319,203,336",67.31,1,"45,200",055550,"45,150","45,363,365,050","1,011,900",신한지주,"474,199,587","44,150","44,200",550,8,,1.28,1.2
8,"21,317,181,157,500","47,158,312",54.09,1,"245,500",005490,"244,500","42,601,962,500","174,978",POSCO,"87,186,835","240,500","242,500","1,500",9,,1.28,0.6
9,"20,841,975,000,000","5,745,073",8.68,1,"319,000",207940,"315,000","17,875,057,500","56,497",삼성바이오로직스,"66,165,000","312,000","313,500","1,500",10,,1.25,0.5


* 해당 데이터는 날짜 데이터가 없기 때문에 post()안에 요청했던 날짜로 date라는 컬럼을 생성하고 날짜를 넣어줍니다. 
* 이 작업도 자동화로 해야겠죠? 위의 코드를 바탕으로 만들어보는 것을 권장합니다.

In [25]:
total_df['date']= '2019-06-27'

* 인덱스 값을 초기화합니다.

In [26]:
total_df.reset_index(drop=True)

,cur_pr_tot_amt,f1,f2,fluc_tp_cd,hgprc,isu_cd,isu_cur_pr,isu_tr_amt,isu_tr_vl,kor_shrt_isu_nm,lst_stk_vl,lwprc,opnprc,prv_dd_cmpr,rn,totCnt,tot_amt_per,updn_rate,date
0,"277,594,888,575,000","3,411,981,118",57.15,1,"46,600",005930,"46,500","583,888,994,023","12,603,534",삼성전자,"5,969,782,550","45,750","46,000",800,1,2398,16.65,1.8,2019-06-27
1,"51,105,766,023,000","367,493,736",50.48,1,"71,600",000660,"70,200","378,736,507,000","5,361,201",SK하이닉스,"728,002,365","69,700","70,100","1,300",2,,3.07,1.9,2019-06-27
2,"31,269,694,600,000","762,330,049",92.64,1,"38,150",005935,"38,000","53,134,817,735","1,399,856",삼성전자우,"822,886,700","37,600","37,600",100,3,,1.88,0.3,2019-06-27
3,"30,020,380,273,500","94,615,472",44.28,2,"142,500",005380,"140,500","48,441,831,000","343,337",현대차,"213,668,187","139,500","141,000","1,000",4,,1.80,-0.7,2019-06-27
4,"26,179,130,280,000","27,239,509",21.23,2,"209,000",068270,"204,000","125,509,765,000","609,613",셀트리온,"128,329,070","203,000","207,500","6,500",5,,1.57,-3.1,2019-06-27
5,"24,672,023,878,500","27,397,742",38.81,1,"350,000",051910,"349,500","59,918,992,000","172,767",LG화학,"70,592,343","343,000","346,000","2,000",6,,1.48,0.6,2019-06-27
6,"22,444,726,437,000","45,640,625",47.89,1,"236,000",012330,"235,500","48,290,490,000","206,434",현대모비스,"95,306,694","228,000","229,000","2,500",7,,1.35,1.1,2019-06-27
7,"21,410,111,353,050","319,203,336",67.31,1,"45,200",055550,"45,150","45,363,365,050","1,011,900",신한지주,"474,199,587","44,150","44,200",550,8,,1.28,1.2,2019-06-27
8,"21,317,181,157,500","47,158,312",54.09,1,"245,500",005490,"244,500","42,601,962,500","174,978",POSCO,"87,186,835","240,500","242,500","1,500",9,,1.28,0.6,2019-06-27
9,"20,841,975,000,000","5,745,073",8.68,1,"319,000",207940,"315,000","17,875,057,500","56,497",삼성바이오로직스,"66,165,000","312,000","313,500","1,500",10,,1.25,0.5,2019-06-27


## 과제
### 배치가 매주 금요일 오후 4시에 실행되기 때문에 5일단위씩 데이터를 수집하는 코드를 작성해 보세요 
* 예) 20190628은 금요일이 때문에 배치가 실행됩니다. 그럼 20190624,20190625, 20190626, 20190627, 20190628 5일치 데이터를 생성하는 코드를 작성